In [ ]:
from importlib import reload
import ipyparallel
import itertools
import numpy as np
import os
from pyprind import prog_percent

from zebrafishframework import ants
from zebrafishframework import io
from zebrafishframework import util

In [ ]:
series_h5 = os.path.abspath('./fish23_6dpf_medium_time.h5')
ts_pregap = np.arange(0, 180)
ts_postgap = np.arange(200, 1800)

# choose representative frames
t_repr_pre = 10
t_repr_post = 240

def mkifnexist(path):
    if not os.path.exists(path):
        os.mkdir(path)

path_processed = os.path.abspath('processed')
path_raw_frames = os.path.join(path_processed, 'raw_frames')
path_post_aligned_to_pre = os.path.join(path_processed, 'post_aligned_to_pre')
path_reg_local = os.path.join(path_processed, 'reg_local')
path_reg_post_pre = os.path.join('processed', 'reg_post_pre')

mkifnexist(path_raw_frames)
mkifnexist(path_processed)
mkifnexist(path_reg_local)
mkifnexist(path_reg_post_pre)
mkifnexist(path_post_aligned_to_pre)

fn_repr_pre = os.path.join(path_processed, 'repr_pre.nrrd')
fn_repr_post = os.path.join(path_processed, 'repr_post.nrrd')

In [ ]:
# run ./startcluster in virtual environment
c = ipyparallel.Client()
view = c.load_balanced_view()

In [ ]:
import ipywidgets
import time

def progress_bar(ar):
    w = ipywidgets.IntProgress()
    w.max = len(ar.msg_ids)
    display(w)
    while not ar.ready():
        w.value = ar.progress
        time.sleep(.1)
    w.value = w.max

In [ ]:
io.save(fn_repr_pre, io.get_frame(series_h5, t_repr_pre))
io.save(fn_repr_post, io.get_frame(series_h5, t_repr_post))

In [ ]:
ants = reload(ants)

In [ ]:
args = ants.AntsArguments(fn_repr_pre, fn_repr_post, output_folder=path_reg_post_pre)

# save some time
args.params[2]['convergences'] = '[100x100x0x0x0,1e-7,10]'

In [ ]:
ar = ants.run_antsreg(args)

In [ ]:
transforms = [
     os.path.abspath(os.path.join(path_reg_post_pre, 'repr_pre_1Warp.nii.gz')), 
     os.path.abspath(os.path.join(path_reg_post_pre, 'repr_pre_0GenericAffine.mat'))]
consts = (series_h5, path_raw_frames, path_post_aligned_to_pre, fn_repr_pre, transforms)
#ts = ts_postgap
ts = np.arange(400, 1800)
l = list(zip(itertools.repeat(consts), ts))

def f(p):
    import os
    from zebrafishframework import io
    from zebrafishframework import ants
    
    consts, t = p
    fn_series, path_raw_frames, path_post_aligned_to_pre, fn_repr_pre, transforms = consts
    
    frame = io.get_frame(fn_series, t)
    fn_frame = os.path.join(path_raw_frames, '%04d.nrrd' % t)
    io.save(fn_frame, frame)
    
    fn_aligned = os.path.join(path_post_aligned_to_pre, '%04d.nrrd' % t)
    ants.apply_transform(fn_frame, fn_repr_pre, fn_aligned, transforms)

In [ ]:
for e in prog_percent(l):
    f(e)